# Matrix non definite problem
Date: 2024-3-16
Author: Zhenjie



## 1. 问题定义
在使用GP库的时候，可能会遇到矩阵非正定的问题，以下是一些可能的解决方法

## 2. 解决方法
### 2.1 GP的负对数似然计算Sigmma时对角线加上x的均值*JITTER

In [ ]:
import torch
JITTER = 1e-6
PI = 3.1415
def negative_log_likelihood(kernel, log_beta, x_train, y_train):
        
    ## A scheme proposed by WJ to make GP more stable -2024/3/4

    y_num, y_dimension = y_train.shape
    Sigma = kernel(x_train, x_train) + log_beta.exp().pow(-1) * torch.eye(
        x_train.size(0)) + JITTER * kernel(x_train, x_train).mean() * torch.eye(x_train.size(0))
    
    L = torch.linalg.cholesky(Sigma)
    #option 1 (use this if torch supports)
    Gamma,_ = torch.triangular_solve(y_train, L, upper = False)
    #option 2
    # gamma = L.inverse() @ Y       # we can use this as an alternative because L is a lower triangular matrix.

    nll =  0.5 * (Gamma ** 2).sum() +  L.diag().log().sum() * y_dimension  \
        + 0.5 * y_num * torch.log(2 * torch.tensor(PI)) * y_dimension
    return -nll

### 2.2 采用更加稳定的GP底层 比如cigp_v10
### 2.3 采用合适的数据标准化或者归一化方法
### 2.4 查看数据中是否有比较偏移均值的点根据实际情况看要不要进行数据剔除
### 2.5 采用log看训练参数以及打印非正定矩阵，肉眼分析问题